In [ ]:
cd ../..

In [ ]:
import glob

import chess
import numpy as np
from scripts.utils import bitboard
from tensorflow import data, keras
from tensorflow.keras import layers

In [ ]:
XX, yy = [], []
for file in glob.glob("data/npz/*.npz"):
    with np.load(file) as f:
        XX.append(f["X"]), yy.append(f["y"])
X, y = np.concatenate(XX), np.concatenate(yy)

In [ ]:
X = X.astype(np.float32)

In [48]:
model = keras.Sequential((
    layers.Conv2D(5, (5, 5), padding="same", activation="relu", input_shape=(8, 8, 6)),
    layers.Conv2D(10, (3, 3), padding="same", activation="relu"),
    layers.Flatten(),
    layers.Dense(100, activation="relu"),
    layers.Dense(1, activation="sigmoid")
))
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 8, 8, 5)           755       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 8, 8, 10)          460       
_________________________________________________________________
flatten_8 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_28 (Dense)             (None, 100)               64100     
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 101       
Total params: 65,416
Trainable params: 65,416
Non-trainable params: 0
_________________________________________________________________


In [49]:
model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["accuracy"]
)

In [ ]:
history = model.fit(
    x=data.Dataset.from_tensor_slices((X, y)).batch(32).repeat(1),
    epochs=2
)

In [50]:
history = model.fit(
    x=X,
    y=y,
    batch_size=32,
    epochs=2,
    validation_split=0.2
)

Epoch 1/2
5236/5236 [==============================] - 33s 6ms/step - loss: 0.0638 - accuracy: 0.0207 - val_loss: 0.0615 - val_accuracy: 0.0288
Epoch 2/2
5236/5236 [==============================] - 35s 7ms/step - loss: 0.0560 - accuracy: 0.0212 - val_loss: 0.0566 - val_accuracy: 0.0290


In [52]:
model.predict(
    np.array([
        bitboard(chess.Board("rnbqkbnr/1ppp1ppp/8/p3p3/2B1P3/5Q2/PPPP1PPP/RNB1K1NR b KQkq - 0 1"))
    ])
)

array([[0.46909174]], dtype=float32)